In [1]:
import os
import mlflow
import mlflow.sklearn
import numpy as np
from imblearn.over_sampling import SMOTE
from mlflow.models.signature import infer_signature
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [3]:
import pandas as pd
df = pd.read_csv("telecom_customer_churn_clean.csv")


In [5]:
df.head()

,Gender,Age,Married,Number of Dependents,City_0,City_1,City_2,City_3,City_4,City_5,...,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Customer Status
0,0,37,1,0,0,0,0,0,0,0,...,1,0,0,1,2,593.30,0.00,0,381.51,0
1,1,46,0,0,0,0,0,0,0,0,...,0,1,1,0,1,542.40,38.33,10,96.21,0
2,1,50,0,0,0,0,0,0,0,0,...,0,0,0,1,1,280.85,0.00,0,134.60,1
3,1,78,1,0,0,0,0,0,0,0,...,1,1,0,1,1,1237.85,0.00,0,361.66,1
4,0,75,1,0,0,0,0,0,0,0,...,1,0,0,1,1,267.40,0.00,0,22.14,1


### Set tracking URI to local folder

In [8]:
mlruns_path = os.path.join(os.getcwd(), "mlruns")
os.makedirs(mlruns_path, exist_ok=True)
mlflow.set_tracking_uri(f"file:///{mlruns_path}")

In [10]:
X = df.drop("Customer Status", axis=1).values
y = df["Customer Status"].values

#### Train-test split 80 / 20
#### Feature scaling

In [13]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=18)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [15]:
experiment_name = "SVM_model"
mlflow.set_experiment(experiment_name)
print(f"Experiment set to: {experiment_name}")

2025/12/09 13:13:53 INFO mlflow.tracking.fluent: Experiment with name 'SVM_model' does not exist. Creating a new experiment.


Experiment set to: SVM_model


#### Running SVM model 1 C value & 1 Kernal value

In [18]:
C_value = 0.001
kernel_type = 'linear'

# mlfow run name
run_name = f"SVM_C={C_value}_kernel={kernel_type}_withoutGS"

with mlflow.start_run(run_name=run_name):

    # Train model
    model = SVC(C=C_value, kernel=kernel_type, probability=True)
    model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)
    
    # Metrics (accuracy , F1 score, precision, recall , loss)
    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall    = recall_score(y_test, y_pred, average='weighted')
    loss      = log_loss(y_test, y_prob)

    # Log param and metric
    mlflow.log_param("C", C_value)
    mlflow.log_param("kernel", kernel_type)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("loss", loss)

    # Model signature
    signature = infer_signature(X_train, model.predict(X_train))
    
    # Log model
    mlflow.sklearn.log_model(sk_model=model, name="svm_model_noGS",
                            signature=signature,
                            input_example=X_train[:5])

    print(f" Run '{run_name}' logged with accuracy: {acc:.4f}")


2025/12/09 13:14:07 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



 Run 'SVM_C=0.001_kernel=linear_withoutGS' logged with accuracy: 0.8300


#### Running SVM model using more than value for parameters

In [21]:
C_values = [0.01, 0.1, 1, 10]
kernels = ['linear', 'rbf', 'poly']
gammas = ['scale', 'auto']

for C in C_values:
    for kernel in kernels:
        for gamma in gammas:

            run_name = f"SVM_C={C}_kernel={kernel}_gamma={gamma}"

            with mlflow.start_run(run_name=run_name):

                # Create model
                model = SVC(C=C, kernel=kernel, gamma=gamma, probability=True)

                # applying 5-fold cross-validation on the training set
                cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
                mean_cv_acc = np.mean(cv_scores)
                
                # Training model
                model.fit(X_train, y_train)
                
                # Evaluate
                y_pred = model.predict(X_test)
                y_prob = model.predict_proba(X_test)
                
                # Metrics (accuracy , F1 score, precision, recall , loss)
                test_acc = accuracy_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred, average='weighted')
                precision = precision_score(y_test, y_pred, average='weighted')
                recall = recall_score(y_test, y_pred, average='weighted')
                loss = log_loss(y_test, y_prob)

                # Model signature
                signature = infer_signature(X_train, model.predict(X_train))

                # Log parameters
                mlflow.log_param("C", C)
                mlflow.log_param("kernel", kernel)
                mlflow.log_param("gamma", gamma)

                # Log metrics
                mlflow.log_metric("cv_accuracy", mean_cv_acc)
                mlflow.log_metric("test_accuracy", test_acc)
                mlflow.log_metric("f1_score", f1)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("loss", loss)

                # Log model
                mlflow.sklearn.log_model(sk_model=model, name="svm_model_GS",
                                         signature=signature,
                                         input_example=X_train[:5])

                
                print(f"Logged: {run_name} | CV Accuracy = {mean_cv_acc:.4f} | Test Accuracy = {test_acc:.4f}")

print("✅ All runs logged successfully!")


Logged: SVM_C=0.01_kernel=linear_gamma=scale | CV Accuracy = 0.8406 | Test Accuracy = 0.8422


Logged: SVM_C=0.01_kernel=linear_gamma=auto | CV Accuracy = 0.8406 | Test Accuracy = 0.8422


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Logged: SVM_C=0.01_kernel=rbf_gamma=scale | CV Accuracy = 0.7128 | Test Accuracy = 0.7307


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Logged: SVM_C=10_kernel=linear_gamma=auto | CV Accuracy = 0.8482 | Test Accuracy = 0.8452


Logged: SVM_C=10_kernel=rbf_gamma=scale | CV Accuracy = 0.8285 | Test Accuracy = 0.8149


Logged: SVM_C=10_kernel=rbf_gamma=auto | CV Accuracy = 0.8289 | Test Accuracy = 0.8149


Logged: SVM_C=10_kernel=poly_gamma=scale | CV Accuracy = 0.8061 | Test Accuracy = 0.7989


Logged: SVM_C=10_kernel=poly_gamma=auto | CV Accuracy = 0.8065 | Test Accuracy = 0.7989
✅ All runs logged successfully!


##### After noticing that linear kernel gives the best acc,
##### Trying more values for C using only linear Kernel

In [24]:
C_values = [0.001, 0.01, 0.1, 1, 0.11, 5, 10, 20, 50, 100, 200]
kernels = ['linear']

for C in C_values:
    for kernel in kernels:
        
        run_name = f"SVM_C={C}_linear"
        with mlflow.start_run(run_name=run_name):

            # Create model
            model = SVC(C=C, kernel=kernel, probability=True)

            # applying 5-fold cross-validation on the training set
            cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
            mean_cv_acc = np.mean(cv_scores)

            # Training model
            model.fit(X_train, y_train)

            # Evaluate
            y_pred = model.predict(X_test)
            y_prob = model.predict_proba(X_test)  

            # Metrics (accuracy , F1 score, precision, recall , loss)
            test_acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            precision = precision_score(y_test, y_pred, average='weighted')
            recall = recall_score(y_test, y_pred, average='weighted')
            loss = log_loss(y_test, y_prob)

            # Model signature
            signature = infer_signature(X_train, model.predict(X_train))

            # Log parameters
            mlflow.log_param("C", C)
            mlflow.log_param("kernel", kernel)

            # Log metrics
            mlflow.log_metric("cv_accuracy", mean_cv_acc)
            mlflow.log_metric("test_accuracy", test_acc)
            mlflow.log_metric("f1_score", f1)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("loss", loss)

            # Log model
            mlflow.sklearn.log_model(sk_model=model, name="svm_model_GS_Linear",
                                     signature=signature,
                                     input_example=X_train[:5])

            
            print(f"Logged: {run_name} | CV Accuracy = {mean_cv_acc:.4f} | Test Accuracy = {test_acc:.4f}")

print("✅ All runs logged successfully!")


Logged: SVM_C=0.001_linear | CV Accuracy = 0.8344 | Test Accuracy = 0.8300


Logged: SVM_C=0.01_linear | CV Accuracy = 0.8406 | Test Accuracy = 0.8422


Logged: SVM_C=0.1_linear | CV Accuracy = 0.8469 | Test Accuracy = 0.8467


Logged: SVM_C=1_linear | CV Accuracy = 0.8488 | Test Accuracy = 0.8452


Logged: SVM_C=0.11_linear | CV Accuracy = 0.8477 | Test Accuracy = 0.8460


...

Logged: SVM_C=10_linear | CV Accuracy = 0.8482 | Test Accuracy = 0.8452


Logged: SVM_C=20_linear | CV Accuracy = 0.8484 | Test Accuracy = 0.8460


Logged: SVM_C=50_linear | CV Accuracy = 0.8484 | Test Accuracy = 0.8460


Logged: SVM_C=100_linear | CV Accuracy = 0.8480 | Test Accuracy = 0.8460


Logged: SVM_C=200_linear | CV Accuracy = 0.8482 | Test Accuracy = 0.8452
✅ All runs logged successfully!


 Appling SMOTE because of the imbalanced data

In [28]:
smote = SMOTE(random_state=94)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

 Using the Balanced data to train model using the same value to observe the difference

In [31]:
C_value = 0.001
kernel_type = 'linear'

# unique run name
run_name = f"SVM_C={C_value}_kernel={kernel_type}_withoutGS_Balanced"

with mlflow.start_run(run_name=run_name):

    # Train model
    model = SVC(C=C_value, kernel=kernel_type, probability=True, class_weight='balanced')
    model.fit(X_train_res, y_train_res)

    # Evaluate
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall    = recall_score(y_test, y_pred, average='weighted')
    loss      = log_loss(y_test, y_prob)

    # Log param and metric
    mlflow.log_param("C", C_value)
    mlflow.log_param("kernel", kernel_type)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("loss", loss)

    signature = infer_signature(X_train_res, model.predict(X_train_res))
    
    # Log model
    mlflow.sklearn.log_model(sk_model=model, name="svm_model_noGS_balanced",
                            signature=signature,
                            input_example=X_train_res[:5])

    print(f" Run '{run_name}' logged with accuracy: {acc:.4f}")


 Run 'SVM_C=0.001_kernel=linear_withoutGS_Balanced' logged with accuracy: 0.7785


 The Balanced data gives less  accuracy 